#Previsione Livelli Ricoveri Terapia Intensiva, Nuovi Positivi e Deceduti per COVID

Nel seguente notebook è stato utilizzato il modello arima ARIMA con i dati resi disponibili dal Ministero della Salute per effettuare un forecast con orizzonte 1, 2, 7 e 14 giorni dei valori considerati.

Sono stati usati i dati regionali resi disponibili via file .csv al [link](https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv) e filtrati per regione di interesse, in questo caso Emilia-Romagna.
Ogni riga è un aggiornamento giornaliero dei seguenti valori: 
data, stato, codice_regione, denominazione_regione, lat, long, ricoverati_con_sintomi, terapia_intensiva,totale_ospedalizzati, isolamento_domiciliare, totale_positivi, variazione_totale_positivi, nuovi_positivi, dimessi_guariti, deceduti, casi_da_sospetto_diagnostico, casi_da_screening, totale_casi, tamponi, casi_testati, note, ingressi_terapia_intensiva, note_test, note_casi, totale_positivi_test_molecolare, totale_positivi_test_antigenico_rapido, tamponi_test_molecolare, tamponi_test_antigenico_rapido, codice_nuts_1, codice_nuts_2. 

Di questi sono state poi utilizzate per l'analisi: 
data, ricoverati_con_sintomi, terapia_intensiva, totale_ospedalizzati, variazione_totale_positivi, nuovi_positivi, deceduti, tamponi, ingressi_terapia_intensiva.

I valori di deceduti e tamponi sono stati differenziati per avere l'incremento giornaliero invece del dato cumulativo che era presente.

L'analisi svolta è su singola variabile.

Dopo la suddivisione è stato effettuato lo scaling attravero un MinMaxScaler che comprime i valori tra -1 ed 1 (la trasformazione viene poi invertita dopo la previsione).

I dati vengono trasformati in lag e valori futuri per ogni timestep dell'intervallo di test. 28 lag vengono usati per calcolare una previsione dell'orizzonte temporale considerato. I parametri del modello vengono calcolati automaticamente tramite autoarima.
(t-n,...,t-1,t) -> (t+1,...,t+p) con n = giorni di lag e p = orizzonte di previsione.

L'errore considerato è il Mean Absolute Error sull'orizzonte di previsione. Viene poi considerata la media dei MAE e MAPE calcolati su tutte le previsioni.

Il grafico mostra l'ultimo slot temporale di previsione.


In [47]:
import os

if 'google.colab' in str(get_ipython()):
    files = ['util.py']

    for file in files:
        os.system('rm ./' + file)
        os.system(
            'wget -nv https://raw.githubusercontent.com/marco-mazzoli/progetto-tesi/master/' + file)

import pandas as pd
import numpy as np
import warnings
from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from numpy.random import seed
import plotly.graph_objects as go

# fix for 'package not found' when installing in Anaconda environment
if 'google.colab' in str(get_ipython()):
    # import pip
    # pip.main(['install', 'pmdarima'])
    !pip install pmdarima

import pmdarima as pm

from xgboost import XGBRegressor
from util import select_relevant_rows, select_attributes, read_movement_data, download_updated_mobility_data, download_updated_mobility_data, save_config, load_config


In [48]:
use_existing_config = True
column_to_predict = 'terapia_intensiva'
columns = ['nuovi_positivi', 'terapia_intensiva', 'deceduti']
split_percent = 0.80
region_focus = 'Emilia-Romagna'
attribute_focus = 'denominazione_regione'
n_futures = [1, 2, 7, 14]

In [49]:
local_region_path = r'../COVID-19/dati-regioni/dpc-covid19-ita-regioni.csv'
remote_region_path = r'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

regions_frame = pd.read_csv(remote_region_path)

region_focus_data = select_relevant_rows(
    regions_frame,
    attribute_focus,
    region_focus
)

frame = select_attributes(region_focus_data, [
    'data',
    'terapia_intensiva',
    'nuovi_positivi',
    'deceduti',
])

frame = pd.DataFrame(frame)
frame['data'] = pd.to_datetime(
    frame['data'])
frame['data'] = frame['data'].dt.strftime(
    r'%Y-%m-%d')
frame['data'] = pd.to_datetime(frame['data'])
# frame = frame[frame['data'] < pd.to_datetime('2022-01-31')]
frame = frame.fillna(0)

frame.rename(columns={'data': 'date'}, inplace=True)
frame.set_index('date', inplace=True)

# revert cumulative data
frame['deceduti'] = frame['deceduti'].diff()

frame.dropna(inplace=True)

# numpy seed
seed(1)

frame.tail()

,terapia_intensiva,nuovi_positivi,deceduti
date,,,
2022-02-03,152,12012,34.0
2022-02-04,149,10779,49.0
2022-02-05,146,8274,37.0
2022-02-06,143,7447,33.0
2022-02-07,144,5203,37.0


In [67]:
def split_series(series, n_past, n_future, arima=False):
    X, y, X_indexes, y_indexes = list(), list(), list(), list()
    index = np.array(series.index).reshape(series.values.shape[0], 1)
    series = series.values

    for window_start in range(len(series)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end > len(series):
            break
        start = 0 if arima == True else window_start

        past, future = series[start:past_end,
                              :], series[past_end:future_end, :]
        past_index, future_index = index[start:past_end,
                                         :], index[past_end:future_end, :]
        X.append(past)
        y.append(future)
        X_indexes.append(past_index)
        y_indexes.append(future_index)

    return np.array(X), np.array(y), np.array(X_indexes), np.array(y_indexes)


def plot_last_results(
    results, column_to_predict):
        pred = results[-1][1][-1]['pred']
        test = results[-1][1][-1]['y_test']

        prediction_trace = go.Scatter(
            x=pred.index, y=pred, mode='lines', name='Prediction')
        truth_trace = go.Scatter(
            x=test.index, y=test, mode='lines', name='Ground Truth')
        layout = go.Layout(
            title=column_to_predict, xaxis={'title': 'Date'},
            yaxis={'title': column_to_predict}, autosize=False, 
            width=600, height=400)
        fig = go.Figure(
            data=[prediction_trace, truth_trace], layout=layout)
        fig.show()

def plot_mape(results, column_to_predict):
    mapes = list(map(lambda x:x[1][1], results))
    dates = list(map(lambda x:x[0], results))

    mapes_trace = go.Scatter(
        x=dates, y=mapes, mode='lines', name='Mape')
    layout = go.Layout(
        title='mape: ' + column_to_predict, xaxis={'title': 'Date'},
        yaxis={'title': 'mape'}, autosize=False, 
        width=600, height=400)
    fig = go.Figure(
        data=[mapes_trace], layout=layout)
    fig.show()


In [51]:
import os, sys

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [68]:
def execute_arima(dataframe, column_to_predict, split_percent, n_future=7):
    df = dataframe[column_to_predict].copy()
    split = int(split_percent*len(df))

    n_past = 28

    test = pd.DataFrame(df[split:])

    scaler = MinMaxScaler(feature_range=(-1, 1))
    s_s = scaler.fit_transform(test.values.reshape(-1, 1))
    s_s = np.reshape(s_s, len(s_s))
    test[column_to_predict] = s_s

    X_test, y_test, X_test_indexes, y_test_indexes = split_series(
        test, n_past, n_future, arima=False)
    
    df_results = []

    for i in range(len(X_test)):
        adf_test = pm.arima.ADFTest()

        diff = 1 if adf_test.should_diff(X_test[i]) else 0

        model = pm.auto_arima(X_test[i],start_p=0,d=diff,start_q=0,
            max_p=5,max_d=5,max_q=5, start_P=0,
            D=diff, start_Q=0, max_P=5,max_D=5,
            max_Q=5, m=12, seasonal=True,
            error_action='ignore',trace=True,
            supress_warnings=True,stepwise=True,
            random_state=20,n_fits=50)
        
        prediction = model.predict(n_periods=n_future)
        prediction = scaler.inverse_transform(prediction.reshape(-1,1))
        y_test[i] = scaler.inverse_transform(y_test[i])

        current = pd.DataFrame(
            {'y_test':y_test[i].reshape(-1),
            'pred':prediction.reshape(-1),
            'dates':y_test_indexes[i].reshape(-1)})

        current.set_index('dates', inplace=True)
        
        df_results.append(current)

    results = {}

    for el in df_results:
        mae = mean_absolute_error(el['y_test'], el['pred'])
        mape = mean_absolute_percentage_error(el['y_test'], el['pred'])
        results[el.index[0]] = (mae, mape, el)

    return list(results.items())


summaries = []

for n_future in n_futures:
    for column_to_predict in columns:
        with HiddenPrints():
            results = execute_arima(
                frame, column_to_predict, split_percent, n_future=n_future)

        summary = ''

        if results is None:
            summary = '|' + column_to_predict + '| seq len ' + str(n_future) + ' no AR found'
        else:
            avg_mae = np.mean(np.array(list(map(lambda x:x[1][0], results))))
            avg_mape = np.mean(np.array(list(map(lambda x:x[1][1], results))))

            summary = '|' + column_to_predict + '| seq len ' + str(n_future) + '| mae: ' + str(avg_mae) + '| mape: ' + str(avg_mape)

        print(summary)
        summaries.append(summary)

        if results is not None:
            if n_future > 2:
                plot_last_results(results, column_to_predict)
            else:
                print(('Pred: ', results[0][1][-1]['pred'].values))
                print(('Test: ', results[0][1][-1]['y_test'].values))
        plot_mape(results, column_to_predict)

print(summaries)



/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning:

divide by zero encountered in reciprocal

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning:

divide by zero encountered in reciprocal

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning:

divide by zero encountered in reciprocal



|nuovi_positivi| seq len 1| mae: 1776.9020767268848| mape: 0.23056667652025
('Pred: ', array([128.00033217]))
('Test: ', array([240.]))


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning:

divide by zero encountered in reciprocal



|terapia_intensiva| seq len 1| mae: 3.8953206744300726| mape: 0.05282434515694693
('Pred: ', array([37.99999989]))
('Test: ', array([33.]))


|deceduti| seq len 1| mae: 6.191204342963424| mape: 375342423248154.44
('Pred: ', array([4.6615724]))
('Test: ', array([2.]))


...

|nuovi_positivi| seq len 14| mae: 3613.292260467613| mape: 0.40018899245003753


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning:

divide by zero encountered in reciprocal



|terapia_intensiva| seq len 14| mae: 10.690800440497513| mape: 0.13764019744575573


|deceduti| seq len 14| mae: 6.636338517852803| mape: 506165391146328.9


['|nuovi_positivi| seq len 1| mae: 1776.9020767268848| mape: 0.23056667652025', '|terapia_intensiva| seq len 1| mae: 3.8953206744300726| mape: 0.05282434515694693', '|deceduti| seq len 1| mae: 6.191204342963424| mape: 375342423248154.44', '|nuovi_positivi| seq len 2| mae: 2156.6385441592975| mape: 0.27839959463285513', '|terapia_intensiva| seq len 2| mae: 4.737023604505905| mape: 0.06473891984999511', '|deceduti| seq len 2| mae: 6.290598740934618| mape: 521507706980447.25', '|nuovi_positivi| seq len 7| mae: 2698.0359924542163| mape: 0.33136653594821275', '|terapia_intensiva| seq len 7| mae: 7.438859337898818| mape: 0.0955024723373089', '|deceduti| seq len 7| mae: 6.550822245084477| mape: 546998821336159.1', '|nuovi_positivi| seq len 14| mae: 3613.292260467613| mape: 0.40018899245003753', '|terapia_intensiva| seq len 14| mae: 10.690800440497513| mape: 0.13764019744575573', '|deceduti| seq len 14| mae: 6.636338517852803| mape: 506165391146328.9']


Risultati:

|nuovi_positivi   | seq len 1 | mae: 1776.90 | mape: 0.23
|nuovi_positivi   | seq len 2 | mae: 2156.63 | mape: 0.27
|nuovi_positivi   | seq len 7 | mae: 2698.03 | mape: 0.33
|nuovi_positivi   | seq len 14| mae: 3613.29 | mape: 0.40
|terapia_intensiva| seq len 1 | mae: 3.89    | mape: 0.05
|terapia_intensiva| seq len 2 | mae: 4.73    | mape: 0.06
|terapia_intensiva| seq len 7 | mae: 7.43    | mape: 0.09
|terapia_intensiva| seq len 14| mae: 10.69   | mape: 0.13
|deceduti         | seq len 1 | mae: 6.19    | mape: 375342423248154.44
|deceduti         | seq len 2 | mae: 6.29    | mape: 521507706980447.25
|deceduti         | seq len 7 | mae: 6.55    | mape: 546998821336159.1
|deceduti         | seq len 14| mae: 6.63    | mape: 506165391146328.9